In [1]:
import os
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing.image import  ImageDataGenerator
from keras.layers import Dense,Flatten,GlobalAveragePooling2D,Input
from keras.models import Model,load_model
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import  ResNet50
from keras.applications.inception_v3 import InceptionV3
import keras.callbacks as kcallbacks
import keras
from keras.preprocessing.image import load_img,img_to_array
import math
import random

Using TensorFlow backend.


# vgg19模型

In [2]:
#os.removedirs('./test/.ipynb_checkpoints')

In [5]:
###########配置的文件####################
TARGET_SIZE=(512,512)
INPUT_SIZE=(512,512,3)
BATCHSIZE=16
##################################
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train/',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        './test/',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')

print('样本0的数量是%s'%len(os.listdir('./train/0')))
print('样本1的数量是%s'%len(os.listdir('./train/1')))
print('样本2的数量是%s'%len(os.listdir('./train/2')))
print('样本3的数量是%s'%len(os.listdir('./train/3')))
print('样本4的数量是%s'%len(os.listdir('./train/4')))
def vgg19net(input_shape,num_class,epochs,savepath='./vgg_model2.hdf5'):
    model_vgg19=VGG19(include_top=False,weights='imagenet',input_shape=input_shape)
    # 冻结所有的训练好的层
    for layer in model_vgg19.layers:  
        layer.trainable = False  
    # 平均池化之后接全连接层
    model= GlobalAveragePooling2D()(model_vgg19.output)  
    model=Dense(1024,activation='relu')(model)
    model=Dense(1024,activation='relu')(model)
    model=Dense(num_class,activation='softmax')(model)
    model_vgg19_clair=Model(model_vgg19.input,model,name='vgg19')
    #定义一下优化的方法
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    #compile一下模型
    model_vgg19_clair.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    #train model
    earlyStopping=kcallbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1, mode='auto')
    saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
#     model_vgg19_clair.fit(X_train_vgg,y_train,batch_size=batchsize,epochs=epochs,verbose=1,
#               validation_data=(X_test_vgg,y_test),shuffle=True,callbacks=[earlyStopping,saveBestModel])
    hist=model_vgg19_clair.fit_generator(
        train_generator,
        steps_per_epoch=math.ceil(49658/BATCHSIZE),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=math.ceil(10000/BATCHSIZE),
        callbacks=[earlyStopping,saveBestModel]
    )
    with open('log.txt','w') as f:
        f.write(str(hist.history))

Found 49658 images belonging to 5 classes.
Found 10000 images belonging to 5 classes.
样本0的数量是21534
样本1的数量是7891
样本2的数量是9752
样本3的数量是5644
样本4的数量是4837


In [6]:
vgg_model=vgg19net(INPUT_SIZE,num_class=5,epochs=20)

Epoch 1/20
3500/3500 [==============================] - 1086s 310ms/step - loss: 1.1882 - acc: 0.5246 - val_loss: 1.0956 - val_acc: 0.5631
Epoch 2/20
3500/3500 [==============================] - 1080s 308ms/step - loss: 1.0999 - acc: 0.5616 - val_loss: 1.0644 - val_acc: 0.5792
Epoch 3/20
3500/3500 [==============================] - 1081s 309ms/step - loss: 1.0717 - acc: 0.5769 - val_loss: 1.0609 - val_acc: 0.5799
Epoch 4/20
3500/3500 [==============================] - 1086s 310ms/step - loss: 1.0581 - acc: 0.5843 - val_loss: 1.0449 - val_acc: 0.5822
Epoch 5/20
3500/3500 [==============================] - 1081s 309ms/step - loss: 1.0441 - acc: 0.5891 - val_loss: 1.0723 - val_acc: 0.5794
Epoch 6/20
3500/3500 [==============================] - 1079s 308ms/step - loss: 1.0395 - acc: 0.5927 - val_loss: 1.0281 - val_acc: 0.5938
Epoch 7/20
3500/3500 [==============================] - 1083s 310ms/step - loss: 1.0316 - acc: 0.5956 - val_loss: 1.1110 - val_acc: 0.5506
Epoch 8/20
3500/3500 [=====

# inceptionv3模型

In [5]:
###########配置的文件####################
TARGET_SIZE=(512,512)
INPUT_SIZE=(512,512,3)
BATCHSIZE=64
##################################
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train/',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        './test/',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')

print('样本0的数量是%s'%len(os.listdir('./train/0')))
print('样本1的数量是%s'%len(os.listdir('./train/1')))
print('样本2的数量是%s'%len(os.listdir('./train/2')))
print('样本3的数量是%s'%len(os.listdir('./train/3')))
print('样本4的数量是%s'%len(os.listdir('./train/4')))
def inception(input_shape,num_class,epochs,savepath='./inception.h5'):
    model_inc=InceptionV3(include_top=False,weights='imagenet',input_shape=input_shape)
    # 冻结所有的训练好的层
    for layer in model_inc.layers:  
        layer.trainable = False  
    # 平均池化之后接全连接层
    model= GlobalAveragePooling2D()(model_inc.output)  
    model=Dense(128,activation='relu')(model)
    model=Dense(num_class,activation='softmax')(model)
    model_inc=Model(model_inc.input,model,name='inception')
    #定义一下优化的方法
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    #compile一下模型
    model_inc.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    #train model
    earlyStopping=kcallbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1, mode='auto')
    saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
#     model_vgg19_clair.fit(X_train_vgg,y_train,batch_size=batchsize,epochs=epochs,verbose=1,
#               validation_data=(X_test_vgg,y_test),shuffle=True,callbacks=[earlyStopping,saveBestModel])
    hist=model_inc.fit_generator(
        train_generator,
        steps_per_epoch=math.ceil(49658/BATCHSIZE),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=math.ceil(10000/BATCHSIZE),
        callbacks=[earlyStopping,saveBestModel],
        workers=8
    )
    with open('inc_log.txt','w') as f:
        f.write(str(hist.history))

Found 49658 images belonging to 5 classes.
Found 10000 images belonging to 5 classes.
样本0的数量是21534
样本1的数量是7891
样本2的数量是9752
样本3的数量是5644
样本4的数量是4837


In [6]:
inception=inception(INPUT_SIZE,num_class=5,epochs=20)

Epoch 1/20
875/875 [==============================] - 688s 786ms/step - loss: 1.0192 - acc: 0.6050 - val_loss: 0.9286 - val_acc: 0.6333
Epoch 2/20
875/875 [==============================] - 677s 774ms/step - loss: 0.9369 - acc: 0.6391 - val_loss: 0.9447 - val_acc: 0.6209
Epoch 3/20
875/875 [==============================] - 677s 774ms/step - loss: 0.9052 - acc: 0.6495 - val_loss: 0.8996 - val_acc: 0.6461
Epoch 4/20
875/875 [==============================] - 678s 775ms/step - loss: 0.8847 - acc: 0.6554 - val_loss: 0.9153 - val_acc: 0.6510
Epoch 5/20
875/875 [==============================] - 678s 775ms/step - loss: 0.8747 - acc: 0.6588 - val_loss: 0.8793 - val_acc: 0.6523
Epoch 6/20
875/875 [==============================] - 677s 774ms/step - loss: 0.8645 - acc: 0.6619 - val_loss: 0.8984 - val_acc: 0.6483
Epoch 7/20
875/875 [==============================] - 679s 776ms/step - loss: 0.8634 - acc: 0.6624 - val_loss: 0.8674 - val_acc: 0.6613
Epoch 8/20
875/875 [============================

# Retnet-50

In [2]:
model_res=ResNet50(include_top=False,weights='imagenet')
model_res.summary()

94658560/94653016 [==============================] - 21s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 6 0           bn_conv1[0][0]                   
___________________________________________

In [3]:
for i, layer in enumerate(model_res.layers):
    print(i, model_res.name)

0 resnet50
1 resnet50
2 resnet50
3 resnet50
4 resnet50
5 resnet50
6 resnet50
7 resnet50
8 resnet50
9 resnet50
10 resnet50
11 resnet50
12 resnet50
13 resnet50
14 resnet50
15 resnet50
16 resnet50
17 resnet50
18 resnet50
19 resnet50
20 resnet50
21 resnet50
22 resnet50
23 resnet50
24 resnet50
25 resnet50
26 resnet50
27 resnet50
28 resnet50
29 resnet50
30 resnet50
31 resnet50
32 resnet50
33 resnet50
34 resnet50
35 resnet50
36 resnet50
37 resnet50
38 resnet50
39 resnet50
40 resnet50
41 resnet50
42 resnet50
43 resnet50
44 resnet50
45 resnet50
46 resnet50
47 resnet50
48 resnet50
49 resnet50
50 resnet50
51 resnet50
52 resnet50
53 resnet50
54 resnet50
55 resnet50
56 resnet50
57 resnet50
58 resnet50
59 resnet50
60 resnet50
61 resnet50
62 resnet50
63 resnet50
64 resnet50
65 resnet50
66 resnet50
67 resnet50
68 resnet50
69 resnet50
70 resnet50
71 resnet50
72 resnet50
73 resnet50
74 resnet50
75 resnet50
76 resnet50
77 resnet50
78 resnet50
79 resnet50
80 resnet50
81 resnet50
82 resnet50
83 resnet50
84

In [2]:
###########配置的文件####################
TARGET_SIZE=(512,512)
INPUT_SIZE=(512,512,3)
BATCHSIZE=64
##################################
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train/',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        './test/',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')
print('样本0的数量是%s'%len(os.listdir('./train/0')))
print('样本1的数量是%s'%len(os.listdir('./train/1')))
print('样本2的数量是%s'%len(os.listdir('./train/2')))
print('样本3的数量是%s'%len(os.listdir('./train/3')))
print('样本4的数量是%s'%len(os.listdir('./train/4')))
def resnet(input_shape,num_class,epochs,savepath='./resnet.h5'):
    model_res=ResNet50(include_top=False,weights='imagenet')
    # 平均池化之后接全连接层
    model= GlobalAveragePooling2D()(model_res.output)  
    model=Dense(128,activation='relu')(model)
    model=Dense(num_class,activation='softmax')(model)
    model_res_clair=Model(model_res.input,model,name='resnet')
    #定义一下优化的方法
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    # fine-tune
    for layer in model_res.layers[:135]:  
        layer.trainable = False 
    for layer in model_res.layers[135:]:  
        layer.trainable = True  
    #compile一下模型
    model_res_clair.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    #train model
    earlyStopping=kcallbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1, mode='auto')
    saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
#     model_vgg19_clair.fit(X_train_vgg,y_train,batch_size=batchsize,epochs=epochs,verbose=1,
#               validation_data=(X_test_vgg,y_test),shuffle=True,callbacks=[earlyStopping,saveBestModel])
    hist=model_res_clair.fit_generator(
        train_generator,
        steps_per_epoch=math.ceil(49658/BATCHSIZE),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=math.ceil(10000/BATCHSIZE),
        callbacks=[earlyStopping,saveBestModel]
    )
    with open('res_net_log.txt','w') as f:
        f.write(str(hist.history))

Found 49658 images belonging to 5 classes.
Found 10000 images belonging to 5 classes.
样本0的数量是21534
样本1的数量是7891
样本2的数量是9752
样本3的数量是5644
样本4的数量是4837


In [3]:
res_model=resnet(INPUT_SIZE,num_class=5,epochs=20)

Epoch 1/20
776/776 [==============================] - 1041s 1s/step - loss: 0.8065 - acc: 0.6889 - val_loss: 3.6835 - val_acc: 0.4276
Epoch 2/20
776/776 [==============================] - 1035s 1s/step - loss: 0.6858 - acc: 0.7350 - val_loss: 0.7388 - val_acc: 0.7097
Epoch 3/20
776/776 [==============================] - 1035s 1s/step - loss: 0.5947 - acc: 0.7732 - val_loss: 0.6813 - val_acc: 0.7401
Epoch 4/20
776/776 [==============================] - 1035s 1s/step - loss: 0.4889 - acc: 0.8174 - val_loss: 0.6614 - val_acc: 0.7499
Epoch 5/20
776/776 [==============================] - 1034s 1s/step - loss: 0.3578 - acc: 0.8712 - val_loss: 0.7818 - val_acc: 0.7422
Epoch 6/20
776/776 [==============================] - 1034s 1s/step - loss: 0.2412 - acc: 0.9133 - val_loss: 0.8844 - val_acc: 0.7570
Epoch 7/20
776/776 [==============================] - 1034s 1s/step - loss: 0.1567 - acc: 0.9446 - val_loss: 0.9621 - val_acc: 0.7273
Epoch 8/20
776/776 [==============================] - 1034s 1s